# GMM

## Load Packages

In [1]:
using Dates, DelimitedFiles, Statistics, LinearAlgebra, Optim, NLsolve

include("jlFiles/printmat.jl")
include("jlFiles/printTable.jl")
include("jlFiles/NWFn.jl")

"""
Create identity matrix (n rows, n columns). In Julia 1.2, use the built-in I(n) instead.
"""
Inn(n) = Matrix(1.0I,n,n)

Inn

# GMM I

This section describes the basic (exactly identified) GMM, that is, when we have as many moment conditions as parameters. (In this case GMM is the same as the classical method of moments, MM.)

The first few cells load functions and data. See further down for the estimation.

In [2]:
x = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)   #start on line 2, column 1
x = x[:,2]         #excess market returns, in %

T = size(x,1)

388

## Traditional Estimation of Mean and Variance

The next cell applies the traditional way of estimating the mean and the variance.

In [3]:
μ  = mean(x)                      #same as setting A*gbar=0
σ² = var(x,corrected=false)       #"false" to use 1/T formula

par_a = [μ;σ²]

printblue("Traditional estimates:")
xx = [par_a [sqrt((σ²/T));sqrt(2*σ²^2/T)]]
colNames = ["coef","std"]
rowNames = ["μ","σ²"]
printTable(xx,colNames,rowNames)

Traditional estimates:
        coef       std
μ      0.602     0.233
σ²    21.142     1.518



## GMM Point Estimates and Distribution

To estimate the mean and variance of $x_{t}$, use the following moment condition

$
\frac{1}{T}\sum\nolimits_{t=1}^{T}g_{t}=0 \: \text{ where } 
$

$
g_{t}(\mu,\sigma^{2})=\left[
\begin{array}
[c]{l}
x_{t}-\mu\\
(x_{t}-\mu)^{2}-\sigma^{2}
\end{array}
\right] .
$

The parameter values ($\mu,\sigma^2$) that make these moment conditions hold must be the same as from the traditional method. In other cases, we may not know the GMM estimates. Therefore, the code below solves for the parameters (and also double checks that they indeed are the same as before).

The distribution of the estimates is

$
\sqrt{T}(\hat{\mu}-\mu_{0})\overset{d}{\rightarrow}N(0,V) 
\: \text{ where } \: 
V = (D_{0}^{\prime}S_{0}^{-1}D_{0})  ^{-1}
$

Clearly, $D_{0}=-\textrm{I}$ and if data is iid then $S_{0}=\text{Var}(g_{t})$.

In [4]:
function Gmm2MomFn(par,x)
    (μ,σ²) = (par[1],par[2])
    g      = [(x .- μ) ((x .- μ).^2 .- σ²)]  #Tx2         
    gbar   = vec(mean(g,dims=1))             #2-element vector
    return g,gbar
end

Gmm2MomFn (generic function with 1 method)

In [5]:
Sol = nlsolve(p->Gmm2MomFn(p,x)[2],par_a)   #solve for the GMM estimates
par_1 = Sol.zero                            

printblue("GMM estimates:")
printmat(par_1)

(g,gbar) = Gmm2MomFn(par_1,x)           #Tx2, moment conditions
printblue("Checking if mean of g_t = 0")
printmat(gbar)

GMM estimates:
     0.602
    21.142

Checking if mean of g_t = 0
     0.000
     0.000



In [6]:
D  = -Inn(2)                 #Jacobian, does not really matter here
S  = NWFn(g,1)               #Newey-West with 1 lag
V1 = inv(D'inv(S)*D)

printblue("GMM estimates:")
xx = [par_1 sqrt.(diag(V1/T))]
printTable(xx,colNames,rowNames)

printstyled("Compare with the traditional estimates",color=:red,bold=true)

GMM estimates:
        coef       std
μ      0.602     0.244
σ²    21.142     2.381

Compare with the traditional estimates

# GMM II

This section expands the GMM calculations by doing an overidentified case: more moment conditions than parameters.

Warning: some of the variables (```g,S,etc```) are overwritten with new values.

## The Moment Conditions

If $x_{t}$ is $N(\mu,\sigma^{2})$, then the following moment conditions should
all be zero (in expectation)

$
g_{t}(\mu,\sigma^{2})=\left[
\begin{array}
[c]{l}
x_{t}-\mu\\
(x_{t}-\mu)^{2}-\sigma^{2}\\
(x_{t}-\mu)^{3}\\
(x_{t}-\mu)^{4}-3\sigma^{4}
\end{array}
\right]  .
$

The first moment condition defines the mean $\mu$, the second defines the
variance $\sigma^{2}$, while the third and forth are the skewness and excess
kurtosis respectively.

In [7]:
function Gmm4MomFn(par,x)
  (μ,σ²) = (par[1],par[2])
  g      = [(x .- μ) ((x .- μ).^2 .- σ²) ((x .- μ).^3) ((x .- μ).^4 .- 3*σ²^2)]    #Tx4
  gbar   = vec(mean(g,dims=1))     #4-element vector
  return g,gbar
end

function DGmm4MomFn(par,x)
    (μ,σ²) = (par[1],par[2])
    D  = [-1                  0    ;                #Jacobian of Gmm4MomFn
          -2*mean(x.-μ)      -1    ;
          -3*mean((x.-μ).^2)   0   ;
          -4*mean((x.-μ).^3)  -6*σ²]
    return D
end    

DGmm4MomFn (generic function with 1 method)

## GMM: Minimizing g'W*g


The following code applies a numerical method to solve a minimization problem with the weighting matrix 

$
W=
\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 0\\
0 & 0 & 0 & 0
\end{bmatrix}
$

The results should be the same (or at least very close to) the previous results, since the $W$ matrix puts all weight on the first two moments (basically mimicking the estimations above). Changing $W$, for instance, by setting $W[3,3]=0.0001$ will give other estimates.

We define the loss function as 

$
\bar{g}'W\bar{g}
$

As a practical matter, it is often the case that a derivative-free method works better than other optimization routines.

In [8]:
function Gmm4MomLossFn(par,x,W=1)
  (g,gbar) = Gmm4MomFn(par,x)
  Loss     = 1.0 + gbar'W*gbar      #to be minimized
  return Loss
end

Gmm4MomLossFn (generic function with 2 methods)

In [9]:
W     = diagm(0=>[1.0,1.0,0.0,0.0])   #weighting matrix, try changing it
#W[3,3] = 0.0001
Sol   = optimize(par->Gmm4MomLossFn(par,x,W),par_a)
par_2 = Optim.minimizer(Sol)

printblue("GMM estimates:")
printmat(par_2)

GMM estimates:
     0.602
    21.142



In [10]:
D  = DGmm4MomFn(par_2,x)               #Jacobian
printblue("The Jacobian is:")
printmat(D)

g,    = Gmm4MomFn(par_2,x)                #Tx4, moment conditions, evaluated at point estimates
S     = NWFn(g,1)                         #variance of sqrt(T)"gbar, NW with 1 lag
V2    = inv(D'W*D)*D'W*S*W'D*inv(D'W*D)

printblue("Weighting matrix")
printmat(W)

printblue("GMM estimates (g'W*g):")
xx = [par_2 sqrt.(diag(V2/T))]
printTable(xx,colNames,rowNames)

The Jacobian is:
    -1.000     0.000
    -0.000    -1.000
   -63.427     0.000
   314.797  -126.854

Weighting matrix
     1.000     0.000     0.000     0.000
     0.000     1.000     0.000     0.000
     0.000     0.000     0.000     0.000
     0.000     0.000     0.000     0.000

GMM estimates (g'W*g):
        coef       std
μ      0.602     0.244
σ²    21.142     2.381



## GMM: A*g = 0


The following code from estimates the parameters (mean and
variance) by combining the 4 original moment conditions in $\bar{g}$ into 2
effective moment conditions, $A\bar{g}$, where $A$ is a $2\times4$ matrix

$
A=\left[
\begin{array}
[c]{cccc}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0
\end{array}
\right]  
$ 

This particular $A$ matrix implies that we use the classical
estimators of the mean and variance. Changing $A$,for instance, setting $A[1,3]=0.001$ will give different estimates.

In [11]:
A = [1 0 0 0;                   #A in A*gbar=0 (here: all weight on first two moments)
     0 1 0 0]                   #try setting A[1,3] = 0.001
            
Sol = nlsolve(p->A*Gmm4MomFn(p,x)[2],par_a)   #solve for the GMM estimates
par_3 = Sol.zero

printblue("GMM estimates:")
printmat(par_3)

(g,gbar) = Gmm4MomFn(par_3,x)        #Tx4, moment conditions. Warning: overwriting old g
q = size(g,2)

printblue("\nChecking if mean of A*g_t = 0")
printmat(A*gbar)

GMM estimates:
     0.602
    21.142


Checking if mean of A*g_t = 0
     0.000
     0.000



In [12]:
D  = DGmm4MomFn(par_3,x)               #Jacobian
printblue("The Jacobian is:")
printmat(D)

S  = NWFn(g,1)
V3 = inv(A*D)*A*S*A'inv(A*D)'

printblue("GMM estimates (A*gbar):")
xx = [par_3 sqrt.(diag(V3/T))]
printTable(xx,colNames,rowNames)

printstyled("Compare with the exactly identified GMM (above)",color=:red,bold=true)

The Jacobian is:
    -1.000     0.000
    -0.000    -1.000
   -63.427     0.000
   314.797  -126.854

GMM estimates (A*gbar):
        coef       std
μ      0.602     0.244
σ²    21.142     2.381

Compare with the exactly identified GMM (above)

# GMM: Minimizing g'Wg, Iterating over W


The following code iterates over the weighting matrix by using $W=S^{-1}$, where

$S = \text{Cov}(\sqrt{T}\bar{g})$ 

is from the previous iteration.

In [13]:
println("\niterated GMM, using optimal weighting matrix, starting with S from previous estimation")

(par_4,par0) = (copy(par_1),copy(par_1))
(Δpar,i)     = (Inf,1)

println("\n\niterating over W starting with the W choice above")
while (Δpar > 1e-3) || (i < 2)    #require at least one iteration
    #global Δpar, par_4, par0, i, W, S    #only needed in REPL/scripts
    local Sol, g
    println("-------iteration  $i, old and new parameters--------")
    W               = inv(S)
    Sol             = optimize(par->Gmm4MomLossFn(par,x,W),par0)   #use last estimates as starting point
    par_4           = Optim.minimizer(Sol)
    printlnPs(par0')
    printlnPs(par_4')
    g,              = Gmm4MomFn(par_4,x)
    S               = NWFn(g,1)
    Δpar            = maximum(abs.(par_4-par0))
    par0            = copy(par_4)             #par0=par_4 would make them always identical
    i               = i + 1
 end

V2 = inv(D'W*D)*D'W*S*W'D*inv(D'W*D)      #if non-optimal weighting matrix
V1 = inv(D'inv(S)*D)                      #with optimal weighting matrix

printblue("\nGMM estimates (g'W*g, iteration over W):")
xx = [par_4 sqrt.(diag(V2/T)) sqrt.(diag(V1/T))]
printTable(xx,[colNames;"std ver. 2"],rowNames,width=12)


iterated GMM, using optimal weighting matrix, starting with S from previous estimation


iterating over W starting with the W choice above
-------iteration  1, old and new parameters--------
     0.602    21.142
     0.877    16.916
-------iteration  2, old and new parameters--------
     0.877    16.916
     0.879    16.648
-------iteration  3, old and new parameters--------
     0.879    16.648
     0.879    16.645
-------iteration  4, old and new parameters--------
     0.879    16.645
     0.879    16.647
-------iteration  5, old and new parameters--------
     0.879    16.647
     0.879    16.647

GMM estimates (g'W*g, iteration over W):
          coef         std  std ver. 2
μ        0.879       0.217       0.217
σ²      16.647       1.311       1.311



In [14]:
printblue("W matrix used in the last iteration, (times 10000)")

printmat(W*10000)

W matrix used in the last iteration, (times 10000)
  1525.564    39.433   -16.963    -0.674
    39.433    18.778    -0.297    -0.050
   -16.963    -0.297     0.306     0.012
    -0.674    -0.050     0.012     0.001

